# Value Function Iteration

### Input Variables:

$\beta$ Firm's Discount Factor  
$\mathbf{P}$ Profits  
  
$\mathbb{\Pi}_D$ Demand Shocks    
$\mathbb{\Pi}_Q$ Quit and Retirement Shock  
  
$\textbf{M}$ Feasibility Indexes 

### Output Variables:

$\mathbf{V}$ Value Function  
$\textbf{G}$ Optimal Policy

## Function:

As of: 29/04/2015

In [1]:
function VFI(β::Float64,P::Array{Array{Float64,2},1},D::Array{Float64,2},Q::SparseMatrixCSC{Float64,Int64},M::Array{Array{Int64,1},1})
    
    """Solve the Dynamic Problem by 
    using Value Function Iteration.

    Parameters
    ----------
    β : Discount Factor
    P : Profits
    D : Transition Matrix of Demand Shocks 
    Q : Transition Matrix of
    M : Matrices with Feasibility Indexes
    
    Output
    ----------
    V : Value Function
    G : Optimal Policy
    """
    
    S_L=length(P);
    S_D=size(D,1);
    
    V0=zeros(S_D,S_L);
    V=ones(S_D,S_L);
    
    while norm(V-V0)>eps(Float32)
        
        V0=deepcopy(V);
        
        Vt=β*D*V0*Q';
        
        for i=1:S_L, j=1:S_D
            
            V[j,i]=maximum(P[i][j,:]+Vt[j,M[i]]);         # 471/541 = 87%
            
        end
        
    end
    
    G=zeros(Int,S_D,S_L);
    
    Vt=β*D*V0*Q';
    
    for i=1:S_L, j=1:S_D
        
        g=findmax([P[i][j,:]+Vt[j,M[i]]...]);
            
        G[j,i]=M[i][g[2]];
            
    end
    
    return V, G
    
end

VFI (generic function with 1 method)

In [2]:
471/541

0.8706099815157117

## Growing Size

$G$

In [3]:
import LabourDP

β=0.9;
N=3;
α₁=0.8;
β₁=0.1;
F_0=0.8;
F_G=1.0;
F̂=1.1;
ϕ=0.2;
S=true;
M_D=10.;
m_D=5.;
ρ=0.8;
N_D=6;
d=2.;
N_G=10;
δ=0.1;
γ=0.5;

## Time

Max Size and Sample Size

In [4]:
Ms=6;
Ss=1;

### Test

In [ ]:
# Warm Up Run
G=3;
W=LabourDP.Wages(G,β₁,α₁);
F=LabourDP.Production(G,F_0,F_G,F̂,div(G,2)+1);
FC=LabourDP.FiringCost(G,ϕ,S,W);
D=LabourDP.Demand(M_D,m_D,ρ,N_D,d);
fP=LabourDP.fPort(G);
L=LabourDP.LabourPortfolio(G,N,N_G,fP);
Feas=LabourDP.FeasibleChoices(L);
QnRFeas=LabourDP.FeasibleQnR(L);
QnR=LabourDP.QnRTransition(L,QnRFeas,δ,γ);
P=LabourDP.Profits(L,D[1],W,F,FC,Feas);
VFI(β,P,D[2],QnR,Feas[2]);#

x=[1:Ms-2];
t=[1.0:Ms-2];
b=[1.0:Ss];

for i=1:Ms-2
            
    G=i+2;
    W=LabourDP.Wages(G,β₁,α₁);
    F=LabourDP.Production(G,F_0,F_G,F̂,div(G,2)+1);
    FC=LabourDP.FiringCost(G,ϕ,S,W);
    D=LabourDP.Demand(M_D,m_D,ρ,N_D,d);
    fP=LabourDP.fPort(G);
    L=LabourDP.LabourPortfolio(G,N,N_G,fP);
    Feas=LabourDP.FeasibleChoices(L);
    QnRFeas=LabourDP.FeasibleQnR(L);
    QnR=LabourDP.QnRTransition(L,QnRFeas,δ,γ);
    P=LabourDP.Profits(L,D[1],W,F,FC,Feas);
    
    x[i]=length(L);
    
    for j=1:Ss

        a=@elapsed VFI(β,P,D[2],QnR,Feas[2])
        b[j]=a;
        
    end
    
    t[i]=minimum(b);
    
end

### Plot Result

In [ ]:
using Gadfly

plot(x=x, y=t,Guide.xlabel("G"),Guide.ylabel("Seconds"),Guide.title("Time"), Geom.line, Geom.point)

## Memory

Max Size and Sample Size

In [ ]:
Ms=6;
Ss=1;

### Test

In [ ]:
# Warm Up Run
G=3;
W=LabourDP.Wages(G,β₁,α₁);
F=LabourDP.Production(G,F_0,F_G,F̂,div(G,2)+1);
FC=LabourDP.FiringCost(G,ϕ,S,W);
D=LabourDP.Demand(M_D,m_D,ρ,N_D,d);
fP=LabourDP.fPort(G);
L=LabourDP.LabourPortfolio(G,N,N_G,fP);
Feas=LabourDP.FeasibleChoices(L);
QnRFeas=LabourDP.FeasibleQnR(L);
QnR=LabourDP.QnRTransition(L,QnRFeas,δ,γ);
P=LabourDP.Profits(L,D[1],W,F,FC,Feas);
VFI(β,P,D[2],QnR,Feas[2]);#

x=[1:Ms-2];
t=[1.0:Ms-2];
b=[1.0:Ss];

for i=1:Ms-2
    
    G=i+2;
    W=LabourDP.Wages(G,β₁,α₁);
    F=LabourDP.Production(G,F_0,F_G,F̂,div(G,2)+1);
    FC=LabourDP.FiringCost(G,ϕ,S,W);
    D=LabourDP.Demand(M_D,m_D,ρ,N_D,d);
    fP=LabourDP.fPort(G);
    L=LabourDP.LabourPortfolio(G,N,N_G,fP);
    Feas=LabourDP.FeasibleChoices(L);
    QnRFeas=LabourDP.FeasibleQnR(L);
    QnR=LabourDP.QnRTransition(L,QnRFeas,δ,γ);
    P=LabourDP.Profits(L,D[1],W,F,FC,Feas);
    
    x[i]=length(L);
    
    for j=1:Ss
        
        a=@allocated VFI(β,P,D[2],QnR,Feas[2])
        b[j]=a/10^9.0;
        
    end
    
    t[i]=minimum(b);
    
end

### Plot Result

In [ ]:
plot(x=x, y=t,Guide.xlabel("G"),Guide.ylabel("Gb"),Guide.title("Memory"), Geom.line, Geom.point)

## Profile

In [ ]:
β=0.9;
G=5;
N=3;
α₁=0.8;
β₁=0.1;
F_0=0.8;
F_G=1.0;
F̂=1.1;
ϕ=0.2;
S=true;
M_D=10.;
m_D=5.;
ρ=0.8;
N_D=6;
d=2.;
N_G=5;
δ=0.1;
γ=0.5;

W=LabourDP.Wages(G,β₁,α₁);
F=LabourDP.Production(G,F_0,F_G,F̂,div(G,2)+1);
FC=LabourDP.FiringCost(G,ϕ,S,W);
D=LabourDP.Demand(M_D,m_D,ρ,N_D,d);
fP=LabourDP.fPort(G);
L=LabourDP.LabourPortfolio(G,N,N_G,fP);
Feas=LabourDP.FeasibleChoices(L);
QnRFeas=LabourDP.FeasibleQnR(L);
QnR=LabourDP.QnRTransition(L,QnRFeas,δ,γ);
P=LabourDP.Profits(L,D[1],W,F,FC,Feas);

In [ ]:
@time VFI(β,P,D[2],QnR,Feas[2])

Sample Size

In [ ]:
Rep=1;

In [ ]:
Profile.clear()
@profile (for i=1:Rep;VFI(β,P,D[2],QnR,Feas[2]);end)
Profile.print()